# 模型推理 - 使用 QLoRA 微调后的 ChatGLM3-6B

In [1]:
import torch 
from transformers import AutoModel,AutoTokenizer,BitsAndBytesConfig
from peft import PeftModel,PeftConfig
model_name_or_path = "THUDM/chatglm3-6b"
peft_model_path = f"models/saved/{model_name_or_path}"

/home/ubuntu/miniconda3/envs/transformers/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-03-26 22:32:43.367937: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-26 22:32:43.370017: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-26 22:32:43.395593: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-26 22:32:43.395617: E external/local_xla/xla/stream_executor/cuda/c

In [2]:
config = PeftConfig.from_pretrained(peft_model_path)

q_config = BitsAndBytesConfig(
    load_in_4bits = True,
    bnb_4bit_quant_type = "nf4",
    bnb_4bit_use_double_quant = True,
    bnb_4bit_compute_dtype = torch.float32
)

base_model = AutoModel.from_pretrained(
    config.base_model_name_or_path,
    quantization_config = q_config,
    trust_remote_code = True,
    device_map = "auto"
)

base_model.requires_grad_(False)
base_model.eval()

ValueError: Can't find 'adapter_config.json' at 'models/saved/THUDM/chatglm3-6b'

## ChatGLM3-6B 

In [ ]:
input_text = '类型#裙*版型#显瘦*风格#文艺*风格#简约*图案#印花*图案#撞色*裙下摆#压褶*裙长#连衣裙*裙领型#圆领'
print(f"输入：\n ${input_text}")
tokenizer = AutoTokenizer.from_pretrained(
    config.base_model_name_or_path,
    trust_remote_code = True
)
response, history = base_model.chat(tokenizer = tokenizer ,query = input_text)
print(f'ChatGLM3-6B 微调前：\n{response}')

In [ ]:
model_peft = PeftModel.from_pretrained(base_model,peft_model_path)
response, history = model_peft.chat(tokenizer=tokenizer, query=input_text)
print(f'ChatGLM3-6B 微调后: \n{response}')